In [1]:
import sys
import os 
from pathlib import Path
import requests
import socket
import pandas as pd
import webbrowser
from requests_oauthlib import OAuth2Session
import gi
import tempfile
gi.require_version('NM', '1.0')
from gi.repository import GLib, NM

In [2]:
sys.path.append(str(Path().absolute().parent))

In [3]:
from eduvpn.oauth2 import one_request, get_open_port
from eduvpn.crypto import gen_code_challenge, gen_code_verifier, common_name_from_cert

# Settings

In [4]:
disco_base_url = 'https://disco.eduvpn.org/v2/'
organisation = 'SURFnet bv'
server = 'Demo'

In [5]:
client_id = "org.eduvpn.app.linux"
scope = ["config"]
code_challenge_method = "S256"

# Utils

In [51]:
def pandas_expand(df: pd.DataFrame, column: str):
    expanded = pd.DataFrame(list(df[column])).add_prefix(column + '_')
    return pd.concat([expanded, df.drop(columns=column)], axis=1)

# organisation list

In [47]:
org_list_url = disco_base_url  + "organization_list.json"
org_list_response = requests.get(org_list_url)

In [48]:
organization_list = pd.DataFrame(org_list_response.json()['organization_list'])

In [49]:
organization_list[:5]

,display_name,org_id,server_list,keyword_list
0,{'da': 'International Business College [TEST]'...,https://wayf.ibc.dk/simplesaml/saml2/idp/metad...,aHR0cHM6Ly93YXlmLmliYy5kay9zaW1wbGVzYW1sL3NhbW...,NaN
1,"{'da': 'Erhvervsakademi Aarhus', 'en': 'Busine...",http://adfs.eaaa.dk/adfs/services/trust,aHR0cDovL2FkZnMuZWFhYS5kay9hZGZzL3NlcnZpY2VzL3...,NaN
2,"{'da': 'KMD DSPARE3', 'en': 'KMD DSPARE3'}",http://dans-support-04.dans-idp-dev01.northeur...,aHR0cDovL2RhbnMtc3VwcG9ydC0wNC5kYW5zLWlkcC1kZX...,NaN
3,"{'da': 'KMD DSUP_PATCH1', 'en': 'KMD DSUP_PATC...",https://dans-idp.kmd.dk:7080,aHR0cHM6Ly9kYW5zLWlkcC5rbWQuZGs6NzA4MA.json,NaN
4,"{'da': 'IT-Supportcentret (ITS) [ADFS]', 'en':...",http://fs4.supportcenter.dk/adfs/services/trust,aHR0cDovL2ZzNC5zdXBwb3J0Y2VudGVyLmRrL2FkZnMvc2...,NaN


In [52]:
org_list_expanded = pandas_expand(organization_list, 'display_name')

In [53]:
surfnet = org_list_expanded[org_list_expanded['display_name_en'] == organisation]
server_list_url = disco_base_url + surfnet['server_list'].iloc[0]

In [54]:
server_list_response = requests.get(server_list_url)

In [56]:
server_list = pd.DataFrame(server_list_response.json()['server_list'])
server_list_exp = pandas_expand(server_list, 'display_name')

In [57]:
server_list_exp

,display_name_da-DK,display_name_en-US,display_name_nb-NO,display_name_nl-NL,display_name_en,base_url,peer_list
0,Holland,The Netherlands,Nederland,Nederland,NaN,https://nl.eduvpn.org/,"[{'base_url': 'https://eduvpn.rash.al/', 'disp..."
1,NaN,NaN,NaN,NaN,Demo,https://demo.eduvpn.nl/,NaN
2,NaN,NaN,NaN,NaN,SURFnet,https://surfnet.eduvpn.nl/,NaN


In [58]:
demo_server = server_list_exp[server_list_exp['display_name_en'] == 'Demo']

In [59]:
demo_base_url = demo_server['base_url'].iloc[0]

In [61]:
info_url = demo_base_url + 'info.json'

In [62]:
info_response = requests.get(info_url)

In [64]:
info = info_response.json()['api']['http://eduvpn.org/api#2']

In [65]:
api_base_uri = info['api_base_uri']
token_endpoint = info['token_endpoint']
authorization_endpoint = info['authorization_endpoint']

In [66]:
port = get_open_port()
redirect_uri = f'http://127.0.0.1:{port}/callback'

In [67]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, auto_refresh_url=token_endpoint, scope=scope)

In [68]:
code_verifier = gen_code_verifier()
code_challenge = gen_code_challenge(code_verifier)
authorization_url, state = oauth.authorization_url(url=authorization_endpoint,
                                                   code_challenge_method=code_challenge_method,
                                                   code_challenge=code_challenge)

In [69]:
webbrowser.open(authorization_url)

True

In [70]:
response = one_request(port, lets_connect=False)

127.0.0.1 - - [28/Apr/2020 17:13:15] "GET /callback?code=eyJhbGciOiJFZERTQSIsInR5cCI6IkpXVCIsImtpZCI6IldMQTA5RWVfWHdoU0ZYalBOZ2ExT3lXRW40a1pUZ2xNR3ppR1RtRnM5MEEifQ.eyJ2Ijo1LCJ0eXBlIjoiYXV0aG9yaXphdGlvbl9jb2RlIiwiYXV0aF9rZXkiOiIyZjQ0OWMyYjY3NDRlYWNjYjBjNjBlYTI3Njg4NGE1NSIsInVzZXJfaWQiOiJodHRwczpcL1wvc2EtZ3cuc3VyZmNvbmV4dC5ubFwvYXV0aGVudGljYXRpb25cL21ldGFkYXRhIWh0dHBzOlwvXC9kZW1vLmVkdXZwbi5ubFwvc2FtbCFlMTEzN2VmNzQyYWRmODMwOGFhYjdlMGE2YjY0M2M4ZTIyY2JmM2ZiIiwiY2xpZW50X2lkIjoib3JnLmVkdXZwbi5hcHAubGludXgiLCJzY29wZSI6ImNvbmZpZyIsInJlZGlyZWN0X3VyaSI6Imh0dHA6XC9cLzEyNy4wLjAuMTo0NTcxMVwvY2FsbGJhY2siLCJjb2RlX2NoYWxsZW5nZSI6InBrM1IxOXQtX2NhUExYYWRQSW1mb0JpRlE5MmRvTjR5MDJ3cUFqQW5mZGciLCJleHBpcmVzX2F0IjoiMjAyMC0wNC0yOFQxNToxODoxNSswMDowMCJ9.m4dNW4kVvXUdLSIknXxbj2XLSHGGyV8ekcOrOs5sm9PCAF9DmSiCdcXL3dh8WWYsrQstz1DvHzqxg7Q_7078Cg&state=f3F0uAztZzDcvPU4PnkRQZ5jZMEHGc HTTP/1.1" 200 -


In [71]:
code = response['code'][0]

In [72]:
assert(state == response['state'][0])

In [73]:
token = oauth.fetch_token(token_url=token_endpoint,
                          code=code,
                          code_verifier=code_verifier,
                          client_id=oauth.client_id,
                          include_client_id=True,
                          )

# profile list

In [74]:
profile_list_response = oauth.get(api_base_uri + '/profile_list')

In [75]:
profile_list_response.json()['profile_list']['data']

[{'profile_id': 'internet', 'display_name': 'internet', 'two_factor': False},
 {'profile_id': 'routes',
  'display_name': 'No rfc1918 routes',
  'two_factor': False}]

In [76]:
response = oauth.post(api_base_uri + '/create_keypair')
keypair = response.json()['create_keypair']['data']
private_key = keypair['private_key']
certificate = keypair['certificate']
common_name = common_name_from_cert(certificate.encode('ascii'))

# profile config

In [77]:
profile_id = 'internet'
response = oauth.get(api_base_uri + f'/profile_config?profile_id={profile_id}')
config = response.text

# check_certificate

In [78]:
response = oauth.get(api_base_uri + f'/check_certificate?common_name={common_name}')
assert(response.json()['check_certificate']['data']['is_valid'])

# system_messages

In [79]:
response = oauth.get(api_base_uri + '/system_messages')

In [80]:
response.json()['system_messages']['data']

[{'type': 'notification',
  'date_time': '2019-05-16T07:59:48Z',
  'message': 'Welcome to the eduVPN demo server by SURFnet. If you have any questions, please let us know by mailing us on eduvpn@surfnet.nl.'}]

# write networkmanager config

In [82]:
tmp = tempfile.NamedTemporaryFile(mode='w+t')
tmp.writelines(config)
tmp.seek(0)
filename = tmp.name

In [83]:
connection = None
for vpn_info in NM.VpnPluginInfo.list_load():
    print("TRY:  plugin %s" % (vpn_info.get_filename()))
    try:
        vpn_plugin = vpn_info.load_editor_plugin()
    except Exception as e:
        print("SKIP: cannot load plugin: %s" % (e))
        continue
    try:
        connection = vpn_plugin.import_(filename)
    except Exception as e:
        print("SKIP: failure to import %s" % (e))
        continue
    break

if connection is None:
    print("None of the VPN plugins was able to import \"%s\"" % (filename))
    sys.exit(1)

TRY:  plugin /usr/lib/NetworkManager/VPN/nm-openvpn-service.name


In [84]:
connection.normalize()

print("connection imported from \"%s\" using plugin \"%s\" (\"%s\", %s)" % (filename, vpn_info.get_filename(), connection.get_id(), connection.get_uuid()))

client = NM.Client.new(None)

main_loop = GLib.MainLoop()

def added_cb(client, result, data):
    try:
        client.add_connection_finish(result)
        print("The connection profile has been successfully added to NetworkManager.")
    except Exception as e:
        print("ERROR: failed to add connection: %s\n" % e)
    main_loop.quit()

client.add_connection_async(connection, True, None, added_cb, None)

main_loop.run()

connection imported from "/tmp/tmphfua3l45" using plugin "/usr/lib/NetworkManager/VPN/nm-openvpn-service.name" ("tmphfua3l45", 8f9fbb32-e740-4796-b811-5e05d49d9a1d)
The connection profile has been successfully added to NetworkManager.
